In [42]:
import numpy as np
import pandas as pd

# Generation of initial population of random antibodies

In [153]:
# m: number of machines
# p: number of parts
# population_size : number of antibody of the initial population
# seed: seed for reproducible results
def generation_initial_population(p, m, population_size, seed):
    MaxCell = min(p,m) #calculation of max number of cells
    number_of_zeros = MaxCell - 1 #number of zeros in each antibody
    antibodies = np.empty(shape=(population_size, p+m+number_of_zeros), dtype=int)
    antibody = np.append(np.array([*range(1,p+m+1)]), np.zeros(number_of_zeros,dtype=int))
    for i in range(0,population_size):
        np.random.seed(seed) 
        np.random.shuffle(antibody) #random positions in the array
        antibodies[i] = antibody
    return antibodies

In [154]:
antibodies = generation_initial_population(p = 7, m = 5, population_size = 3, seed = 1995)
antibodies

array([[ 1,  8, 11,  0,  0, 12,  0,  9, 10,  0,  6,  2,  4,  7,  3,  5],
       [ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1, 10, 12, 11,  0,  8,  0,  0,  5,  0,  2,  9,  3,  7,  6,  4]])

# Evaluate all existing antibodies and compute their affinities

Lectura de los datos del problema + Traducción del formato del dataset a matriz trabajo-estación

In [165]:
def part_machine_incidence_matrix(data):    
    f = open(data,'r')
    lines = [line.split('\n') for line in f]
    f.close()

    m,p = [int(num) for num in lines[8][0].split(' ')] #m: number of machines, p: number of parts

    machines=[[m,p]]
    for i in range(9,9+m):
        #machines.append([int(num) for num in lines[i][0].split(' ')])
        machines.append([int(lines[i][0].split(' ')[j]) for j in range(1,m)])

    columns, rows = ['M'+str(i) for i in range(1,m+1)], ['P'+str(i) for i in range(1,p+1)]
    m_p_matrix = pd.DataFrame(columns= columns, index= rows)

    ones_zeros = []
    for i in range(1,len(machines)):
        aux = []
        for j in range(1,p+1):
            if j in machines[i]: aux.append(1)
            else: aux.append(0)
        ones_zeros.append(aux)

    for i in range(0,len(columns)):
        m_p_matrix[columns[i]] = ones_zeros[i]

    return m_p_matrix, m, p

In [166]:
matrix, m, p = part_machine_incidence_matrix('data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt')
matrix

,M1,M2,M3,M4,M5
P1,1,0,0,1,0
P2,0,1,0,1,1
P3,0,1,1,1,0
P4,0,1,1,1,1
P5,1,1,1,0,1
P6,1,0,1,0,1
P7,1,0,0,0,0


## Decodificación de anticuerpos
1er Paso: Separación de celdas.

In [157]:
def cell_identification(antibodies):
    total_cells = []
    for antibodie in antibodies:
        print("antibodie", antibodie)
        flag = 0 #bandera que indica si el num anterior es un cero
        cells, cell = [],[]
        i = 0
        for num in antibodie:
            if num == 0 or num == antibodie[len(antibodie)-1]: 
                if flag == 0: 
                    i=i+1
                    cells.append(cell)
                    # print(cell)
                    cell = []
                flag = 1
            else: 
                cell.append(num)
                # print(num)
                flag = 0
        total_cells.append(cells)
        # print("cells", cells)
    # print("list of all cells for all antibodies",total_cells)
    # print("cells for antibody 1",total_cells[0])
    return total_cells


In [158]:
cell_identification(antibodies = antibodies)

antibodie [ 1  8 11  0  0 12  0  9 10  0  6  2  4  7  3  5]
antibodie [ 1  9  6  3  4  2  7 10  0  5 12  8  0  0 11  0]
antibodie [ 1 10 12 11  0  8  0  0  5  0  2  9  3  7  6  4]


[[[1, 8, 11], [12], [9, 10], [6, 2, 4, 7, 3]],
 [[1, 9, 6, 3, 4, 2, 7, 10], [5, 12, 8], [11]],
 [[1, 10, 12, 11], [8], [5], [2, 9, 3, 7, 6]]]

reorganizar filas y columnas de la matriz maquinas-trabajos en función de lo descrito por el anticuerpo

# Select N% of antibodies with highest affinities

# Clone the selected antibodies